In [31]:
import numpy as np
import pandas as pd
# packages to read ra and dec ; convert b/w hrs and deg easily
from astropy.coordinates import SkyCoord, FK4
import astropy.units as u
import time
import pickle

In [32]:
# read the given text file, using separator as '|'
df = pd.read_csv('grbdata27052022.txt', skiprows = 4, sep = '|')

# remove the first unnecessary column 'Unnamed : 0'
df.drop(['Unnamed: 0', 'Unnamed: 33'], inplace = True, axis = 1) 

# remove the extra spaces in the column names
df.columns = [x.strip() for x in df.columns.values]

# make a new column concatenating 'ra' and 'dec'
df['RD'] = df['ra'] + df['dec']
df.RD

# there are some NaN values in RD columns
# removing all the rows which have NaN in RD column
df = df.dropna(axis = 0, subset = ['RD'])

# resetting index
df.reset_index(drop = True, inplace = True)

In [33]:
# numpy array of SkyCoord tuple (ra, dec)
radec = np.array([SkyCoord(x, frame = FK4, unit = (u.hourangle, u.deg), obstime = "J2000") for x in df['RD']])

# add two new columns with ra and dec values 
df['ra_val'] = np.array([x.ra.value for x in radec])
df['dec_val'] = np.array([x.dec.value for x in radec])

In [34]:
# saving the cleaned data frame into a file
df.to_csv('gbmdatacleaned.csv')

In [35]:
R = 10 # radius in degrees

In [36]:
def nearbyGRB(n, ra_val, dec_val):
    '''
    find nearbyGRB in a 'R' degree radius
    n = index (center of circle)
    ra_val = ra values , dec_val = dec values
    '''
    nbd = []
    ra = ra_val[n]
    dec = dec_val[n]
    for i in range(n+1, len(ra_val)):
        if ra_val[i] == ra or dec_val[i] == dec:
            continue
        if((ra_val[i] - ra)**2 + (dec_val[i] - dec)**2 <= R**2):
            nbd.append(i)
    return nbd

In [37]:
start = time.time()

nbd = []
for i in range(len(df['ra_val'])):
    ls = nearbyGRB(i, df['ra_val'], df['dec_val'])
    if ls:
        nbd.append((i,ls))

end = time.time()
print('time elapsed : {}'.format(end - start))

time elapsed : 73.1350269317627


In [39]:
# pickling nbd array
with open('nbd_array', 'wb') as f:
    pickle.dump(nbd, f)